In [1]:
from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2025/11/19 03:55:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/11/19 03:55:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [4]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-NYtaxi-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1763377572298, experiment_id='1', last_update_time=1763377572298, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
runs

[<Run: data=<RunData: metrics={'rmse': 5.8218039768395995}, params={'learning_rate': '0.05402678187363913',
  'max_depth': '34',
  'min_child_weight': '1.2554721710445258',
  'objective': 'reg:linear',
  'reg_alpha': '0.04662372244915497',
  'reg_lambda': '0.12306624810536808',
  'seed': '42'}, tags={'mlflow.runName': 'adorable-crab-62',
  'mlflow.source.name': '/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'codespace',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-NYtaxi-zoomcamp/02-experiment-tracking/mlruns/1/6d1183df87414a22aeaff8e19855b5af/artifacts', end_time=1763381041007, experiment_id='1', lifecycle_stage='active', run_id='6d1183df87414a22aeaff8e19855b5af', run_name='adorable-crab-62', start_time=1763380932631, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=None>,
 <Run: data=<RunData: metrics

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 6d1183df87414a22aeaff8e19855b5af, rmse: 5.8218
run id: 9fdb187557974616b4acd058a81cb7d6, rmse: 5.8456
run id: 5403e1c99d794f7d83b0bb45b0cf4662, rmse: 5.8482
run id: 9519cc33c295434bb2274f11255cd3ff, rmse: 5.8669
run id: 567f507bf61a4e9090f71ff10282025b, rmse: 5.9014


In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
run_id = "101e505075af42abb305428a3ef479e6"
mlflow.register_model(model_uri="m-446ba6741b144bfea647c289d2dd7bac", name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1763525802463, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1763525802463, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id=None, run_link=None, source='m-446ba6741b144bfea647c289d2dd7bac', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [13]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_6443/3294275207.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [14]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_6443/1718116654.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1763525802463, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1763525951511, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id=None, run_link=None, source='m-446ba6741b144bfea647c289d2dd7bac', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [ ]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

In [15]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": np.sqrt(mean_squared_error(y_test, y_pred))}